In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

In [ ]:
class_names = ["buildings" , "forest" , "glacier" , "mountain" , "sea" , "street"]
class_names_label = {class_names:i for i , class_names in enumerate (class_names) }
size = (150,150)
class_names_label

In [ ]:
def load_data(size=(150, 150)):
    datasets = [
        "/kaggle/input/intel-image-classification/seg_train/seg_train",
        "/kaggle/input/intel-image-classification/seg_test/seg_test"
    ]
    output = []
    
    for dataset in datasets:
        images = []
        labels = []
        print("Loading {}".format(dataset))
        
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            folder_path = os.path.join(dataset, folder)
            
            for file in tqdm(os.listdir(folder_path)):
                img_path = os.path.join(folder_path, file)
                image = cv2.imread(img_path)
                
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, size)
                
                images.append(image)
                labels.append(label)
        
        images = np.array(images, dtype="float32")
        labels = np.array(labels, dtype="int32")
        output.append((images, labels))
    
    return output

In [ ]:
(train_images,train_labels),(test_images,test_labels) = load_data()

In [ ]:
train_images , train_labels = shuffle(train_images,train_labels,random_state=32)

In [ ]:
print("the train label shape is :" , train_labels.shape)
print("the test label shape is :" , test_labels.shape)

In [ ]:
import pandas as pd

_, train_counts = np.unique(train_labels, return_counts=True)
_, test_counts = np.unique(test_labels, return_counts=True)
pd.DataFrame({'train': train_counts,
                    'test': test_counts}, 
             index=class_names
            ).plot.bar()
plt.show()

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Input(shape=(150, 150, 3)))  # Add Input layer as the first layer
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())  # Instantiate the Flatten layer
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))  # Add activation function for the output layer

model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

In [ ]:
model.compile(optimizer = "adam" , loss = "sparse_categorical_crossentropy" ,  metrics=['accuracy'] )
history = model.fit(train_images,train_labels , batch_size=128, epochs=10, validation_split = 0.2)